# 哈希表的基本操作
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 哈希表（Hash）

### 基本功能

Redis的Hash被设计出来，就是为了存储大量的键值对映射。储存相同数量的键值映射，Hash所占用的内存空间，远远小于字符串。

### 基本语法

```python
import redis

client = redis.Redis()

# 向Hash表中添加一个键值对
client.hset('Key', '字段名', '值')

# 向Hash表中添加多个键值对
client.hmset('Key', {'字段名1': '值1', '字段名2': '值2', '字段名3': '值3'})

# 查询字段是否在哈希表中
client.hexists('Key', '字段名')

# 查询哈希表中一个有多少个字段
client.hlen('Key')

# 获取Hash表里面所有的字段名（慎用）
client.hkeys('Key')

# 读取一个字段中的数据
client.hget('Key', '字段名')

# 读取多个字段中的数据
client.hmget('Key', ['字段名1', '字段名2', '字段名3'])

# 读取全部字段（慎用）
client.hgetall('Key')
```

## 向哈希表中写入数据

In [32]:
# 初始化Redis连接
import redis
import json
client = redis.Redis()

In [33]:
# 添加一条数据

info = json.dumps({'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'})
client.hset('user', 10001, info)

1

In [34]:
# 插入多条数据
info_dict = {
    10002: json.dumps({'name': '王小三', 'age': 27, 'salary': 10000, 'address': '浙江'}),
    10003: json.dumps({'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}),
    10004: json.dumps({'name': '刘小五', 'age': 30, 'salary': 990, 'address': '武汉'})
}
client.hmset('user', info_dict)

True

In [35]:
# 字段名不一定非要是数字，也可以是字母或者中文，字段值的数据类型也可以任意设定

client.hset('user', '马小七', 780)

1

## 检查字段信息

In [36]:
# 检查字段是否在Hash表中
client.hexists('user', 10003)

True

In [37]:
client.hexists('user', '马小七')

True

In [38]:
client.hexists('user', '不存在的字段')

False

In [40]:
client.hexists('不存在的Key', 10003)

False

In [41]:
# 查询Hash中有多少字段
client.hlen('user')

5

In [43]:
keys = client.hkeys('user')
for key in keys:
    print(key.decode())

10003
马小七
10001
10002
10004


## 获取键值对


In [44]:
# 获取单条数据
data = client.hget('user', '马小七')
print(f'返回的数据类型为：{type(data)}, 数据内容为：{data}')
print(f'数据解析以后为：{data.decode()}')


返回的数据类型为：<class 'bytes'>, 数据内容为：b'780'
数据解析以后为：780


In [45]:
data = client.hget('user', 10003)
data_dict = json.loads(data)
print(f'用JSON解析以后：{data_dict}')

用JSON解析以后：{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}


In [46]:
# 批量获取数据

data_list = client.hmget('user', [10001, 10003])
for data in data_list:
    print(json.loads(data))

{'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'}
{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}


In [47]:
## 获取全部数据

all_data = client.hgetall('user')
print(f'先来看看返回的数据是什么样的：{all_data}')



先来看看返回的数据是什么样的：{b'10003': b'{"name": "\\u85cf\\u5c0f\\u56db", "age": 16, "salary": 10, "address": "\\u56db\\u5ddd"}', b'\xe9\xa9\xac\xe5\xb0\x8f\xe4\xb8\x83': b'780', b'10001': b'{"name": "\\u5f20\\u5c0f\\u4e8c", "age": 18, "salary": 100, "address": "\\u5317\\u4eac"}', b'10002': b'{"name": "\\u738b\\u5c0f\\u4e09", "age": 27, "salary": 10000, "address": "\\u6d59\\u6c5f"}', b'10004': b'{"name": "\\u5218\\u5c0f\\u4e94", "age": 30, "salary": 990, "address": "\\u6b66\\u6c49"}'}


In [48]:
for field, value in all_data.items():
    print(f'字段名：{field.decode()}, 值：{json.loads(value)}')

字段名：10003, 值：{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}
字段名：马小七, 值：780
字段名：10001, 值：{'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'}
字段名：10002, 值：{'name': '王小三', 'age': 27, 'salary': 10000, 'address': '浙江'}
字段名：10004, 值：{'name': '刘小五', 'age': 30, 'salary': 990, 'address': '武汉'}


![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)